In [2]:
import os
import math
import numpy as np
from operator import xor
import readFloatsFile
from JidtWrapper import JidtWrapper, np2JArr

# Create a TE wrapper class
jidtWrapper1 = JidtWrapper()

### Analytical transfer entropy:

* $T(A\rightarrow B) = I(Y_t ; X_{t \in [-1, -L]} | Y_{t \in [-1, -L]})$
  - The information shared by current Y, and history of X of length L, given that the history of Y of length L is known

Additional concepts
* https://github.com/jlizier/jidt

In [31]:
# Shift all elements of the array to the right cyclically
def cyclicShift(arr, num):
    return np.hstack((arr[-num:], arr[:-num]))

# Shift all elements of the array to the right, replacing them with zeros
def lossyShift(arr, num):
    return np.hstack(([0]*num, arr[:-num]))

aaa = np.linspace(1, 10, 10)
print("original array:      ", aaa)
print("lossy-shifted by 2:  ", lossyShift(aaa, 2))
print("cyclic-shifted by 2: ", cyclicShift(aaa, 2))

original array:       [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
lossy-shifted by 2:   [0. 0. 1. 2. 3. 4. 5. 6. 7. 8.]
cyclic-shifted by 2:  [ 9. 10.  1.  2.  3.  4.  5.  6.  7.  8.]


### Example 1: TE for Two-Partite Binary Data

In [54]:
# Generate some random binary data.
sourceArray1 = np.random.randint(0, 2, 100)
sourceArray2 = np.random.randint(0, 2, 100)
destArray1 = lossyShift(sourceArray1, 1).astype(int)
destArray2 = lossyShift(sourceArray1, 2).astype(int)

# Create a TE calculator and run it:
param = {'historyLength' : 2, 'kernelWidth' : 1}
te_random = jidtWrapper1.calcTwoPartiteTE_Discrete(sourceArray2, destArray1, param)
te_lag_1 = jidtWrapper1.calcTwoPartiteTE_Discrete(sourceArray1, destArray1, param)
te_lag_2 = jidtWrapper1.calcTwoPartiteTE_Discrete(sourceArray1, destArray2, param)
te_lag_1_rev = jidtWrapper1.calcTwoPartiteTE_Discrete(destArray1, sourceArray1, param)
te_lag_2_rev = jidtWrapper1.calcTwoPartiteTE_Discrete(destArray2, sourceArray1,param)

print("Uncorrelated: %.4f" % te_random)
print("1 step lag  : %.4f" % te_lag_1)
print("2 step lag  : %.4f" % te_lag_2)
print("1 step lag, backwards  : %.4f" % te_lag_1_rev)
print("2 step lag, backwards  : %.4f" % te_lag_1_rev)

Uncorrelated: 0.0018
1 step lag  : 0.9490
2 step lag  : 0.0184
1 step lag, backwards  : 0.0110
2 step lag, backwards  : 0.0110


### Example 2: TE for Multidimensional Two-Partite Binary Data

In [6]:
nData = 100

# First row is random
# 2nd row is the first row offset to the right by 1
rowData = np.zeros((2, nData), dtype=int)
rowData[0] = np.random.randint(0, 2, nData)
rowData[1] = np.hstack((rowData[0, nData-1], rowData[0, 0:nData-1]))

# Add observations of transfer across one cell to the right per time step:
param = {'historyLength' : 2, 'kernelWidth' : 1}
result2D = jidtWrapper1.calcTwoPartiteTE_Discrete(rowData, 1, param)
nObserv = jidtWrapper1.javaClass.getNumObservations()
print(('The result should be close to 1 bit here, since we are executing copy ' + \
      'operations of what is effectively a random bit to each cell here: %.3f ' + \
      'bits from %d observations') % (result2D, nObserv))

The result should be close to 1 bit here, since we are executing copy operations of what is effectively a random bit to each cell here: 0.996 bits from 100 observations


### Example 3: TE for Continuous Data Kernel

For copied source, should give something close to 1 bit: Expected correlation is expected covariance / product of expected standard deviations: (where square of destArray standard dev is sum of squares of std devs of underlying distributions)

In [56]:
# Generate some random normalised data.
numObservations = 1000
covariance = 0.4

# Make two uncorrelated normal-distributed sources
sourceArray  = np.random.normal(0, 1, numObservations)
sourceArray2 = np.random.normal(0, 1, numObservations)

# Destination array of random normals with partial correlation to previous value of sourceArray
destArray = np.zeros(numObservations)
destArray[1:] += covariance * sourceArray[:-1]
destArray[1:] += (1 - covariance) * np.random.normal(0, 1, numObservations-1)

# Normalise the individual variables
# Use history length 1 (Schreiber k=1), kernel width of 0.5 normalised units
param = {'method': 'TE_KERNEL', 'initParam': [1, 0.5], 'properties': {"NORMALISE" : "true"}}
result1 = jidtWrapper1.runJavaTwoPartite((sourceArray, destArray), param)
result2 = jidtWrapper1.runJavaTwoPartite((sourceArray2, destArray), param)

corr_expected = covariance / (1 * np.sqrt(covariance**2 + (1-covariance)**2))
rez_expected = -0.5*np.log(1-np.power(corr_expected, 2)) / np.log(2)
print("TE result %.4f bits; expected to be close to %.4f bits for these correlated Gaussians but biased upwards" % (result1, rez_expected))
print("TE result %.4f bits; expected to be close to 0 bits for uncorrelated Gaussians but will be biased upwards" % result2)

TE result 0.3522 bits; expected to be close to 0.2653 bits for these correlated Gaussians but biased upwards
TE result 0.0886 bits; expected to be close to 0 bits for uncorrelated Gaussians but will be biased upwards


### Example 4: TE for Continuous Data Kraskov

Note that the calculation is a random variable (because the generated data is a set of random variables) - the result will be of the order of what we expect, but not exactly equal to it; in fact, there will be a large variance around it. Expected correlation is expected covariance / product of expected standard deviations: (where square of destArray standard dev is sum of squares of std devs of underlying distributions)

Perform calculation with a correlated and an uncorrelated source

In [58]:
# Generate some random normalised data.
numObservations = 1000
covariance = 0.4

# Make two uncorrelated normal-distributed sources
sourceArray  = np.random.normal(0, 1, numObservations)
sourceArray2 = np.random.normal(0, 1, numObservations)

# Destination array of random normals with partial correlation to previous value of sourceArray
destArray = np.zeros(numObservations)
destArray[1:] += covariance * sourceArray[:-1]
destArray[1:] += (1 - covariance) * np.random.normal(0, 1, numObservations-1)

# Create a TE calculator and run it:
# Use Kraskov parameter K=4 for 4 nearest points
param = {'method': 'TE_KRASKOV', 'initParam': [1], 'properties': {"NORMALISE": "true", "k": "4"}}
result1 = jidtWrapper1.runJavaTwoPartite((sourceArray, destArray), param)   
result2 = jidtWrapper1.runJavaTwoPartite((sourceArray2, destArray), param)

corr_expected = covariance / (1 * np.sqrt(covariance**2 + (1-covariance)**2))
rez_expected = -0.5*np.log(1-np.power(corr_expected, 2))
print("TE result %.4f nats; expected to be close to %.4f nats for these correlated Gaussians" % (result1, rez_expected))
print("TE result %.4f nats; expected to be close to 0 nats for these uncorrelated Gaussians" % result2)

TE result 0.2153 nats; expected to be close to 0.1839 nats for these correlated Gaussians
TE result -0.0034 nats; expected to be close to 0 nats for these uncorrelated Gaussians


### Example 5: TE Binary Multivariate Transfer

In [14]:
# Generate some random binary data.
numRows = 2
numObservations = 100
sourceArray1 = np.random.randint(0, 2, numRows*numObservations).reshape((numObservations, numRows))
sourceArray2 = np.random.randint(0, 2, numRows*numObservations).reshape((numObservations, numRows))

# Destination variable takes a copy of the first bit of the source in bit 1,
#  and an XOR of the two bits of the source in bit 2:
destArray = np.zeros((numObservations, numRows), dtype=int)
destArray[1:, 0] = sourceArray1[:-1, 0]
destArray[1:, 1] = xor(sourceArray1[:-1, 0], sourceArray1[:-1, 1])

# We need to construct the joint values of the dest and source before we pass them in,
#  and need to use the matrix conversion routine when calling from Matlab/Octave:
sourceArray1Combined = jidtWrapper1.computeCombinedValues(sourceArray1)
sourceArray2Combined = jidtWrapper1.computeCombinedValues(sourceArray2)
destArrayCombined    = jidtWrapper1.computeCombinedValues(destArray)

param = {'historyLength' : 4, 'kernelWidth' : 1}
result1 = jidtWrapper1.calcTwoPartiteTE_Discrete(sourceArray1Combined, destArrayCombined, param)
result2 = jidtWrapper1.calcTwoPartiteTE_Discrete(sourceArray2Combined, destArrayCombined, param)

print('For source which the 2 bits are determined from, result should be close to 2 bits : %.3f' % result1)
print('For random source, result should be close to 0 bits in theory: %.3f' % result2)
print('The result for random source is inflated towards 0.3 due to finite observation length (%d).'
      'One can verify that the answer is consistent with that from a random source by checking:'
      'teCalc.computeSignificance(1000); ans.pValue\n' % numObservations)

For source which the 2 bits are determined from, result should be close to 2 bits : 1.922
For random source, result should be close to 0 bits in theory: 0.396
The result for random source is inflated towards 0.3 due to finite observation length (100).One can verify that the answer is consistent with that from a random source by checking:teCalc.computeSignificance(1000); ans.pValue



### Example 6: Dynamic Calling Mutual Info

In [20]:
datafile = os.path.join(os.getcwd(), 'data/4ColsPairedNoisyDependence-1.txt')
data = np.array(readFloatsFile.readFloatsFile(datafile))

MI_ESTIMATOR_LIST = ["MI_MV_KRASKOV", "MI_MV_KERNEL", "MI_MV_Gaussian"]
for miEst in MI_ESTIMATOR_LIST:
    paramUniVar = {'method': miEst, 'initParam': [1, 1], 'properties': {}, 'NO_CONV_JARRAY' : True}
    paramJoint  = {'method': miEst, 'initParam': [2, 2], 'properties': {}, 'NO_CONV_JARRAY' : True}
    miUniVar = jidtWrapper1.runJavaTwoPartite((data[:, 0], data[:, 2]), paramUniVar)
    miJoint  = jidtWrapper1.runJavaTwoPartite((data[:, [0, 1]], data[:, [2, 3]]), paramJoint)

    print("Estimator:", miEst,"; MI of two univariate variables=%.3f" % miUniVar, ", joint 2D variables=%.3f" % miJoint)

Estimator: MI_MV_KRASKOV ; MI of two univariate variables=0.100 , joint 2D variables=0.364
Estimator: MI_MV_KERNEL ; MI of two univariate variables=0.159 , joint 2D variables=1.120
Estimator: MI_MV_Gaussian ; MI of two univariate variables=0.089 , joint 2D variables=0.297


### Example 9 - Transfer entropy on continuous data using Kraskov estimators with auto-embedding

Transfer entropy (TE) calculation on continuous-valued data using the Kraskov-estimator TE calculator, with automatic selection of embedding parameters 

In [27]:
# Examine the heart-breath interaction that Schreiber originally looked at:
datafile = 'data/SFI-heartRate_breathVol_bloodOx.txt'
data = np.array(readFloatsFile.readFloatsFile(datafile))

# Select data points 2350:3550, pulling out the relevant columns:
breathRate = data[2350:3551,1]; 
heartRate = data[2350:3551,0];


# Set properties for auto-embedding of both source and destination using the Ragwitz criteria:
#  a. Auto-embedding method
#  b. Search range for embedding dimension (k) and delay (tau)
teCalcClass = jidtWrapper1.getJavaClass("TE_KRASKOV")
param = {'method': 'TE_KRASKOV', 'initParam': (), 'properties': {
    teCalcClass.PROP_AUTO_EMBED_METHOD: teCalcClass.AUTO_EMBED_METHOD_RAGWITZ,
    teCalcClass.PROP_K_SEARCH_MAX: "6",
    teCalcClass.PROP_TAU_SEARCH_MAX: "6"
}}

teBreathToHeart = jidtWrapper1.runJavaTwoPartite((breathRate, heartRate), param)  

# Check the auto-selected parameters and print out the result:
optimisedK    = int(jidtWrapper1.javaClass.getProperty(teCalcClass.K_PROP_NAME))
optimisedKTau = int(jidtWrapper1.javaClass.getProperty(teCalcClass.K_TAU_PROP_NAME))
optimisedL    = int(jidtWrapper1.javaClass.getProperty(teCalcClass.L_PROP_NAME))
optimisedLTau = int(jidtWrapper1.javaClass.getProperty(teCalcClass.L_TAU_PROP_NAME))

print("TE(breath->heart) was %.3f nats for " % teBreathToHeart, end='')
print("(heart embedding:) k=%d, k_tau=%d, " % (optimisedK, optimisedKTau))
print("(breath embedding:) l=%d,l_tau=%d, " % (optimisedL, optimisedLTau), end='')
print("optimised via Ragwitz criteria \n")



# Next, embed the destination only using the Ragwitz criteria:
# Since we're only auto-embedding the destination, we supply
#  source embedding here (to overwrite the auto embeddings from above):
param = {'method': 'TE_KRASKOV', 'initParam': (), 'properties': {
    teCalcClass.PROP_AUTO_EMBED_METHOD: teCalcClass.AUTO_EMBED_METHOD_RAGWITZ_DEST_ONLY,
    teCalcClass.PROP_K_SEARCH_MAX: "6",
    teCalcClass.PROP_TAU_SEARCH_MAX: "6",
    teCalcClass.L_PROP_NAME: "1",
    teCalcClass.L_TAU_PROP_NAME: "1"
}}

teBreathToHeartDestEmbedding = jidtWrapper1.runJavaTwoPartite((breathRate, heartRate), param) 

# Check the auto-selected parameters and print out the result:
optimisedK    = int(jidtWrapper1.javaClass.getProperty(teCalcClass.K_PROP_NAME))
optimisedKTau = int(jidtWrapper1.javaClass.getProperty(teCalcClass.K_TAU_PROP_NAME))

print("TE(breath->heart) was %.3f nats for " % teBreathToHeart, end='')
print("(heart embedding:) k=%d, k_tau=%d, " % (optimisedK, optimisedKTau))
print("optimised via Ragwitz criteria, plus (breath embedding:) l=1,l_tau=1")


TE(breath->heart) was 0.052 nats for (heart embedding:) k=2, k_tau=1, 
(breath embedding:) l=5,l_tau=1, optimised via Ragwitz criteria 

TE(breath->heart) was 0.052 nats for (heart embedding:) k=2, k_tau=1, 
optimised via Ragwitz criteria, plus (breath embedding:) l=1,l_tau=1
